In [15]:
import requests
from bs4 import BeautifulSoup
import math
import pandas
import re


resultList = []

url = "https://search.naver.com/search.naver?"

params = {
    "where": 'news',

    # 네이버 기사 검색 값
    "query": '삼성전자',

    # 페이지네이션 값
    "start": 0,

    # "nso": 'so:r,p:1y,a:all'
}
# nso: so: r, p: 1y, a: all -> 최근 1년
# nso: so: r, p: 6m, a: all -> 최근 6개월
# nso: so: r, p: 1d, a: all -> 1일
# 없으면 전체 검색

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'}
raw = requests.get(url, headers=header, params=params)
html = BeautifulSoup(raw.text, "html.parser")

# 검색결과 html body
articles = html.select("ul.type01 > li")

# 전체 기사 수
html_select = html.select("#main_pack > section.sc_new.sp_nnews._fe_news_collection._prs_nws > div.api_subject_bx > div.group_news > ul > li")
totalCount = for html_select[0].text.split(' / ')[1][:-1]

for i in range(0, math.floor(int(totalCount)/10)+1):
    if i == 0:
        params['start'] = i
    else: 
        params['start'] = i * 10 + 1

    raw = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, params=params)
    html = BeautifulSoup(raw.text, "html.parser")
    articles = html.select("ul.type01 > li")

    for ar in articles:
        # 제목 값
        title = ar.select_one("a._sp_each_title").text

        # 검색된 기사의 url을 가져와서 다시 html을 get
        articleUrl = ar.find("a")["href"]
        innerRaw = requests.get(articleUrl, headers={'User-Agent': 'Mozilla/5.0'})

        # 가져온 기사 html중 '기사', '@' string을 모두 가져온다
        innerHtml = BeautifulSoup(innerRaw.text, "html.parser")
        reporter = innerArticles = innerHtml(text=re.compile("기자"))
        reporterEmail = innerArticles = innerHtml(text=re.compile("@"))

        # 언론사 값
        source = ar.select_one("span._sp_each_source").text

        # 등록일 값
        date = ar.select_one("dd.txt_inline").text.split("  ")[1]

        res = {"title": title, "company": source,
               "url": articleUrl, "date": date, "reporter": reporter, "reporterEmail": reporterEmail}
        resultList.append(res)

# 검색된 기사 갯수
resultList.append({"totalCount": totalCount})

df = pandas.DataFrame(resultList)

df.to_csv('blockChain_articles.csv')
df.to_excel('blockChain_articles.xlsx')

10
<li class="bx" id="sp_nws1"> <div class="news_wrap api_ani_send"> <div class="news_area"> <div class="news_info"> <div class="api_save_group _keep_wrap"> <a aria-pressed="false" class="btn_save _keep_trigger" data-url="https://www.chosun.com/international/us/2024/04/15/PRNCZ7JZR5BMDNK2OU5VEAPHEM/?utm_source=naver&amp;utm_medium=referral&amp;utm_campaign=naver-news" href="#" onclick="tCR('a=nws*h.kep&amp;r=1&amp;i=880000ED_000000000000000003828456&amp;u=javascript'); return false;" role="button"><i class="spnew ico_save">문서 저장하기</i></a> <div class="api_ly_save _keep_save_layer"> <a class="spnew_af item item_save _keep_save" data-cr-off="a=nws*h.kepoff&amp;r=1&amp;i=880000ED_000000000000000003828456&amp;u=javascript" data-cr-on="a=nws*h.kepon&amp;r=1&amp;i=880000ED_000000000000000003828456&amp;u=javascript" href="#" role="button">Keep에 저장</a> <a class="spnew_af item item_quick" href="https://keep.naver.com/" onclick="return goOtherCR(this,'a=nws*h.kephome&amp;r=1&amp;i=880000ED_000000

IndexError: list index out of range